In [ ]:
!pip install vncorenlp
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import json
from vncorenlp import VnCoreNLP
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
raw_dataset_law_train = pd.read_json("/kaggle/input/datasetlawvn/train.json")
raw_dataset_law_val = pd.read_json("/kaggle/input/datasetlawvn/val.json")

In [3]:
raw_dataset_law = pd.concat([raw_dataset_law_train, raw_dataset_law_val], ignore_index=True)

In [4]:
fields_dataset_with_subquestion_raw = raw_dataset_law[['subtopic', 'field']]

In [5]:
fields_dataset_with_question_raw = raw_dataset_law[['question', 'field']]

In [6]:
import pandas as pd
from tqdm import tqdm

fields_dataset = fields_dataset_with_subquestion_raw

final_df = pd.DataFrame(columns=['Index', 'Subtopic', 'Field'])

for i in tqdm(range(len(fields_dataset)), desc="Processing"):
    subtopic_data = fields_dataset['subtopic'].iloc[i]
    field_subtopic = fields_dataset['field'].iloc[i]
    
    subtopic_data = {key: value for key, value in subtopic_data.items() if value is not None}
    
    subtopic_df = pd.DataFrame(list(subtopic_data.items()), columns=['Index', 'Subtopic'])
    
    field_df = pd.DataFrame({'Field': [field_subtopic] * len(subtopic_df)})
    
    result_df = pd.concat([subtopic_df, field_df], axis=1)
    
    final_df = pd.concat([final_df, result_df], ignore_index=True)

print(final_df)

Processing: 100%|██████████| 36070/36070 [02:09<00:00, 278.12it/s]

       Index                                           Subtopic  \
0          0  Khi doanh nghiệp trong nước hợp tác với nhà đầ...   
1          1  Nhà đầu tư nước ngoài được áp dụng điều kiện t...   
2          2  Khi nhà đầu tư nước ngoài và nhà đầu tư trong ...   
3          0  Phẫu thuật làm mỏm cụt ngón được hiểu như thế ...   
4          1  Khi nào người bệnh được phẫu thuật làm mỏm cụt...   
...      ...                                                ...   
111585     1  Thực hiện cấp lại giấy phép sử dụng tần số và ...   
111586     2  Trình tự cấp lại giấy phép sử dụng tần số và t...   
111587     0  Việc trẻ em khuyết tật bị tách tời khỏi cha mẹ...   
111588     1  Cha mẹ bị hạn chế quyền đối với con là trẻ em ...   
111589     2  Những ai có quyền yêu cầu Tòa án hạn chế quyền...   

                      Field  
0                    Đầu tư  
1                    Đầu tư  
2                    Đầu tư  
3           Thể thao - Y tế  
4           Thể thao - Y tế  
...            

In [7]:
final_df.rename(columns={'Subtopic': 'question'}, inplace=True)
final_df.rename(columns={'Field': 'field'}, inplace=True)
final_df

,Index,question,field
0,0,Khi doanh nghiệp trong nước hợp tác với nhà đầ...,Đầu tư
1,1,Nhà đầu tư nước ngoài được áp dụng điều kiện t...,Đầu tư
2,2,Khi nhà đầu tư nước ngoài và nhà đầu tư trong ...,Đầu tư
3,0,Phẫu thuật làm mỏm cụt ngón được hiểu như thế ...,Thể thao - Y tế
4,1,Khi nào người bệnh được phẫu thuật làm mỏm cụt...,Thể thao - Y tế
...,...,...,...
111585,1,Thực hiện cấp lại giấy phép sử dụng tần số và ...,Công nghệ thông tin
111586,2,Trình tự cấp lại giấy phép sử dụng tần số và t...,Công nghệ thông tin
111587,0,Việc trẻ em khuyết tật bị tách tời khỏi cha mẹ...,Quyền dân sự
111588,1,Cha mẹ bị hạn chế quyền đối với con là trẻ em ...,Quyền dân sự


In [8]:
fields_dataset_raw = pd.concat([final_df, fields_dataset_with_question_raw], ignore_index=True)
fields_dataset_raw

,Index,question,field
0,0,Khi doanh nghiệp trong nước hợp tác với nhà đầ...,Đầu tư
1,1,Nhà đầu tư nước ngoài được áp dụng điều kiện t...,Đầu tư
2,2,Khi nhà đầu tư nước ngoài và nhà đầu tư trong ...,Đầu tư
3,0,Phẫu thuật làm mỏm cụt ngón được hiểu như thế ...,Thể thao - Y tế
4,1,Khi nào người bệnh được phẫu thuật làm mỏm cụt...,Thể thao - Y tế
...,...,...,...
147655,NaN,Kho bạc Nhà nước tham gia giao dịch công cụ nợ...,Chứng khoán
147656,NaN,Tôi nhờ Ban tư vấn hướng dẫn giúp tôi về trình...,Đầu tư
147657,NaN,Cho tôi hỏi đối tượng phục vụ của Trung tâm tư...,Lao động - Tiền lương
147658,NaN,Cho tôi hỏi thực hiện cấp lại giấy phép sử dụn...,Công nghệ thông tin


In [9]:
fields_dataset_raw.shape

(147660, 3)

In [10]:
fields_dataset_raw.to_csv('fields_dataset_raw.csv', index=False, encoding='utf-8')

In [11]:
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [13]:
segmenter_fields_dataset = fields_dataset_raw.copy()
segmenter_fields_dataset['question'] = segmenter_fields_dataset['question'].apply(lambda x: ' '.join([' '.join(y) for y in rdrsegmenter.tokenize(x)]))
segmenter_fields_dataset['field'] = segmenter_fields_dataset['field'].apply(lambda x: ' '.join([' '.join(y) for y in rdrsegmenter.tokenize(x)]))

In [14]:
segmenter_fields_dataset

,Index,question,field
0,0,Khi doanh_nghiệp trong nước hợp_tác với nhà_đầ...,Đầu_tư
1,1,Nhà_đầu_tư nước_ngoài được áp_dụng điều_kiện t...,Đầu_tư
2,2,Khi nhà_đầu_tư nước_ngoài và nhà_đầu_tư trong ...,Đầu_tư
3,0,Phẫu_thuật làm mỏm cụt ngón được hiểu như_thế_...,Thể_thao - Y_tế
4,1,Khi nào người_bệnh được phẫu_thuật làm mỏm cụt...,Thể_thao - Y_tế
...,...,...,...
147655,NaN,Kho_bạc Nhà_nước tham_gia giao_dịch công_cụ nợ...,Chứng_khoán
147656,NaN,Tôi nhờ Ban tư_vấn hướng_dẫn giúp tôi về trình...,Đầu_tư
147657,NaN,Cho tôi hỏi đối_tượng phục_vụ của Trung_tâm tư...,Lao_động - Tiền_lương
147658,NaN,Cho tôi hỏi thực_hiện cấp lại giấy_phép sử_dụn...,Công_nghệ_thông_tin


In [15]:
fields_raw  = segmenter_fields_dataset['field'].apply(lambda x: [x])

In [16]:
fields_raw

0                        [Đầu_tư]
1                        [Đầu_tư]
2                        [Đầu_tư]
3               [Thể_thao - Y_tế]
4               [Thể_thao - Y_tế]
                   ...           
147655              [Chứng_khoán]
147656                   [Đầu_tư]
147657    [Lao_động - Tiền_lương]
147658      [Công_nghệ_thông_tin]
147659             [Quyền dân_sự]
Name: field, Length: 147660, dtype: object

In [17]:
all_fields = []
for field in fields_raw:
    if field is not None:
        all_fields.extend(field)

len(set(all_fields))

26

In [18]:
multilabel_fields = MultiLabelBinarizer()

In [19]:
fields_mlb = multilabel_fields.fit_transform(fields_raw)

In [20]:
fields_mlb

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [21]:
fields_mlb.shape

(147660, 26)

In [22]:
fields = pd.DataFrame(fields_mlb, columns=multilabel_fields.classes_)

In [23]:
fields

,Bảo_hiểm,Bất_động_sản,Bộ_máy hành_chính,Chứng_khoán,Công_nghệ_thông_tin,Doanh_nghiệp,Dịch_vụ pháp_lý,Giao_thông - Vận_tải,Giáo_dục,Kế_toán - Kiểm_toán,...,Thể_thao - Y_tế,Thủ_tục Tố_tụng,Tiền_tệ - Ngân_hàng,Trách_nhiệm hình_sự,Tài_chính nhà_nước,Vi_phạm hành_chính,Văn_hoá - Xã_hội,Xuất_nhập_khẩu,Xây_dựng - Đô_thị,Đầu_tư
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147655,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
147656,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
147657,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
147658,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
datalaw_fields = pd.concat([segmenter_fields_dataset['question'], fields], axis=1)

In [25]:
datalaw_fields

,question,Bảo_hiểm,Bất_động_sản,Bộ_máy hành_chính,Chứng_khoán,Công_nghệ_thông_tin,Doanh_nghiệp,Dịch_vụ pháp_lý,Giao_thông - Vận_tải,Giáo_dục,...,Thể_thao - Y_tế,Thủ_tục Tố_tụng,Tiền_tệ - Ngân_hàng,Trách_nhiệm hình_sự,Tài_chính nhà_nước,Vi_phạm hành_chính,Văn_hoá - Xã_hội,Xuất_nhập_khẩu,Xây_dựng - Đô_thị,Đầu_tư
0,Khi doanh_nghiệp trong nước hợp_tác với nhà_đầ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Nhà_đầu_tư nước_ngoài được áp_dụng điều_kiện t...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,Khi nhà_đầu_tư nước_ngoài và nhà_đầu_tư trong ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Phẫu_thuật làm mỏm cụt ngón được hiểu như_thế_...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,Khi nào người_bệnh được phẫu_thuật làm mỏm cụt...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147655,Kho_bạc Nhà_nước tham_gia giao_dịch công_cụ nợ...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
147656,Tôi nhờ Ban tư_vấn hướng_dẫn giúp tôi về trình...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
147657,Cho tôi hỏi đối_tượng phục_vụ của Trung_tâm tư...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
147658,Cho tôi hỏi thực_hiện cấp lại giấy_phép sử_dụn...,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
fields_columns = datalaw_fields.columns

In [27]:
fields_columns

Index(['question', 'Bảo_hiểm', 'Bất_động_sản', 'Bộ_máy hành_chính',
       'Chứng_khoán', 'Công_nghệ_thông_tin', 'Doanh_nghiệp', 'Dịch_vụ pháp_lý',
       'Giao_thông - Vận_tải', 'Giáo_dục', 'Kế_toán - Kiểm_toán',
       'Lao_động - Tiền_lương', 'Lĩnh_vực khác', 'Quyền dân_sự',
       'Sở_hữu_trí_tuệ', 'Thuế - Phí - Lệ_Phí', 'Thương_mại',
       'Thể_thao - Y_tế', 'Thủ_tục Tố_tụng', 'Tiền_tệ - Ngân_hàng',
       'Trách_nhiệm hình_sự', 'Tài_chính nhà_nước', 'Vi_phạm hành_chính',
       'Văn_hoá - Xã_hội', 'Xuất_nhập_khẩu', 'Xây_dựng - Đô_thị', 'Đầu_tư'],
      dtype='object')

In [28]:
import json
from tqdm import tqdm

data = datalaw_fields.to_dict(orient='records')

final_json = []

for item in tqdm(data, desc="Processing"):
    row = {}
    for column in fields_columns:
        row[column] = item.get(column, 0)
    final_json.append(row)

with open('fields_datalaw.json', 'w', encoding='utf-8') as json_file:
    json.dump(final_json, json_file, ensure_ascii=False, indent=3)

Processing: 100%|██████████| 147660/147660 [00:01<00:00, 84714.48it/s]
